In [2]:
import math
import hashlib
import inspect
import json
import pandas as pd
import numpy as np
import re
from hashlib import sha256
from collections import Counter
import string
import os
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import spacy
from spacy.matcher import Matcher
from spacy.lang.en import English
from spacy.matcher import Matcher
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score

In [3]:
df_train = pd.read_csv('data/covid19_data.csv')
df_test_pred = pd.read_csv('data/covid19_unlabelled_test.csv')

In [54]:
df_train.shape #(5531, 15)
df_train.head()

,verifiedby,country,class,title,published_date,country1,country2,country3,country4,article_source,ref_source,source_title,content_text,category,lang,label
0,Delfi Melo Detektorius (Lie Detector),Lithuania,FALSE,Claims that coronavirus is fake and Belarus i...,2020/05/11,Lithuania,NaN,NaN,NaN,https://www.delfi.lt/news/melo-detektorius/mel...,poynter,Baltarusija atšventė nesisaugodama nuo koronav...,Gegužės 9-ąją Baltarusijos sostinėje Minske su...,NaN,lt,false
1,AfricaCheck,"United States, South Africa",False,Muammar Gaddafi predicted the current coronav...,2020/03/21,United States,South Africa,NaN,NaN,https://africacheck.org/fbcheck/no-gaddafi-did...,poynter,"No, Gaddafi didn’t predict coronavirus, but di...",As the world struggles to bring the new corona...,NaN,en,false
2,AFP,Hong Kong,False,Video shows quarantined people in a building ...,2020/02/17,Hong Kong,NaN,NaN,NaN,http://u.afp.com/QuarantineChina,poynter,"This video shows tower blocks in Shanghai, not...","It has been viewed more than 200,000 times., T...",NaN,en,false
3,FactCrescendo,India,False,"A video shows a massive explosion in Wuhan, C...",2020/02/17,India,NaN,NaN,NaN,https://english.factcrescendo.com/2020/02/17/t...,poynter,"This blast took place in Tiajin, China in 2015...","The fact behind every news!, Ever since, the W...",NaN,en,false
4,Newtral.es,"Spain, Peru, United States, Mexico",False,Chlorine dioxide cures the coronavirus.,2020/03/25,Spain,Peru,United States,Mexico,https://www.newtral.es/bulo-dioxido-de-cloro-c...,poynter,El dióxido de cloro no cura el coronavirus; al...,Circula un video de Youtube titulado: “El Coro...,NaN,es,false


In [56]:
df_train.published_date.value_counts()

2020/03/17             102
2020/03/23             100
2020/04/02              99
2020/03/20              98
2020/03/19              96
                      ... 
2020-01-31 00:00:00      1
2020-03-24 00:00:00      1
2020-03-25 00:00:00      1
2020-04-30 00:00:00      1
2020-02-20 00:00:00      1
Name: published_date, Length: 246, dtype: int64

In [43]:
class_value = df_train['class'].value_counts(dropna=False)

In [60]:
df_train['class'].unique()

array(['FALSE', 'False', 'Correct Attribution', 'True', 'Explanatory',
       'false', 'Misleading', 'MISLEADING', 'Mostly True', 'No Evidence',
       'partly false', 'Mostly false', 'Partly false', 'PARTLY FALSE',
       'No evidence', 'Mostly False', 'misleading', 'Labeled Satire',
       'News', 'Partly False', 'Misleading/False', 'Mixture',
       'no evidence', 'MOSTLY FALSE', 'Misattributed', 'Unproven',
       'Miscaptioned', 'Fake', 'Half True', 'PARTLY TRUE', 'Not true',
       'Scam', "(Org. doesn't apply rating)", 'Partially false',
       'MOSTLY TRUE', 'Partly true', 'mislEADING', 'NO EVIDENCE',
       'half true', 'false and misleading', 'mostly false', nan,
       'HALF TRUE', 'Two Pinocchios', 'Suspicions', 'Partly FALSE',
       'PANTS ON FIRE', 'Correct', 'Misinformation / Conspiracy theory',
       'IN DISPUTE', 'HALF TRUTH', 'MiSLEADING', 'Partially correct',
       'Unlikely', 'Fake news', 'True but', 'Mostly true', 'Collections',
       'Mixed', 'Unverified', 'Pa

In [52]:
class_value.index
class_value.head(20).sum() #5416 - 98%
class_value.head(40)

False                          2110
FALSE                          2079
Misleading                      463
MISLEADING                      173
Mostly false                    101
Partly false                     99
misleading                       80
No evidence                      57
Mostly False                     43
Mixture                          41
True                             25
Explanatory                      24
No Evidence                      24
News                             23
PARTLY FALSE                     22
Unproven                         12
MOSTLY FALSE                     11
Partly False                     10
Miscaptioned                     10
partly false                      9
mostly false                      9
Mostly True                       8
MOSTLY TRUE                       7
Misattributed                     7
(Org. doesn't apply rating)       7
HALF TRUE                         6
Correct Attribution               5
Partially false             

In [39]:
df_train['label'] = df_train['class'].replace({'False' : 'false',
'FALSE' : 'false',
'Misleading' : 'false',
'MISLEADING' : 'false',
'Mostly false' : 'false',
'Partly false': 'true',
'misleading' : 'false',
'No evidence' : 'true',
'Mostly False' : 'false',
'Mixture': 'tbd',
'True': 'true',
'Explanatory': 'true',
'No Evidence': 'true',
'News': 'true',
'PARTLY FALSE': 'false',
'Unproven': 'tbd',
'MOSTLY FALSE': 'false',
'Partly False': 'false',
'Miscaptioned': 'tbd',
'partly false': 'false',
'mostly false' : 'false',
'Mostly True': 'true',
'MOSTLY TRUE': 'true',
'Misattributed': 'tbd',
"(Org. doesn't apply rating)": 'tbd',
'HALF TRUE': 'true',
'Correct Attribution' :'true',
'Partially false': 'false',
'Labeled Satire': 'tbd',
'Fake' : 'false',
'NO EVIDENCE': 'true',
'false' : 'false',
'Two Pinocchios': 'tbd',
'Scam' : 'false',
'no evidence': 'true',
'Half True': 'true',
'PARTLY TRUE': 'true',
'half true': 'true',
'Correct': 'true',
'mislEADING' : 'false',
'Suspicions': 'false',
'Not true' : 'false',
'nan': 'tbd',
'Partly FALSE': 'false',
'Misleading/False' : 'false',
'PANTS ON FIRE': 'tbd',
'Partially true': 'true',
'Mixed': 'tbd',
'IN DISPUTE': 'tbd',
'Unverified': 'tbd',
'HALF TRUTH': 'true',
'Collections': 'tbd',
'Partially correct': 'true',
'MiSLEADING' : 'false',
'Mostly true': 'true',
'True but': 'true',
'false and misleading' : 'false',
'Partly true': 'true',
'Misinformation / Conspiracy theory' : 'false',
'Unlikely' : 'false',
'Fake news' : 'false'})

In [40]:
df_train['label'].value_counts(dropna=False)

false    5140
true      299
tbd        90
NaN         2
Name: label, dtype: int64

In [44]:
df_train['category'].value_counts(dropna=False)

NaN                             5306
US Government Response            36
Conspiracy Theories               22
Trump and the Pandemic II         20
Memes and Misinformation II       19
Origins and Spread                19
Business and Industry             17
Prevention and Treatments II      15
Viral Videos                      15
International Response            15
Prevention and Treatments         10
Trump and the Pandemic            10
Memes and Misinformation          10
Entertainment & Media              9
Prophecies and Predictions         8
Name: category, dtype: int64

In [45]:
df_train['lang'].value_counts(dropna=False)

en       2069
es        961
tl        398
pt        335
fr        288
NaN       250
hi        149
it        119
de         70
uk         64
zh-tw      64
hr         63
te         60
ar         58
id         53
mk         50
mr         41
ml         41
ta         39
pl         38
lt         34
ja         33
ko         32
ru         32
da         31
nl         30
bn         27
gu         21
lv         16
et         14
pa         13
ur         11
fa          9
th          7
ne          4
sv          3
tr          1
sk          1
vi          1
fi          1
Name: lang, dtype: int64

In [46]:
df_train['ref_source'].value_counts(dropna=False)

poynter    5306
snopes      225
Name: ref_source, dtype: int64

In [49]:
df_train['country1'].value_counts(dropna=False)

 India            1062
 United States     632
 Spain             376
 Brazil            308
NaN                225
                  ... 
 Qatar               1
 Tuvalu              1
 Oman                1
 Congo               1
 Rwanda              1
Name: country1, Length: 109, dtype: int64

In [50]:
df_train['verifiedby'].value_counts(dropna=False)

 AFP                     515
 FactCrescendo           324
 PolitiFact              262
snopes                   225
 Maldita.es              210
                        ... 
 La Voz de Guanacaste      2
 FactCheckNI               1
 Faktabaari/FactBar        1
 Periodismo de Barrio      1
 Mygopen                   1
Name: verifiedby, Length: 96, dtype: int64